In [1]:
import mwparserfromhell
import numpy as np
import pandas as pd
import re
from mw.xml_dump import Iterator
from mw import Timestamp

## helpers

In [2]:
def multiple_replace(text):
    
    adict = {'[[': '', ']]': '', ' ': '_'}
    
  # Create a regular expression from all of the dictionary keys
    regex = re.compile("|".join(map(re.escape, adict.keys(  ))))

  # For each match, look up the corresponding value in the dictionary
    sub_text = regex.sub(lambda match: adict[match.group(0)], text)

    first_title = re.split("\|", sub_text)
    
    return first_title[0]

In [3]:
def aux_fun(x):
    try:
        return Timestamp(np.int64(x))
    except:
        print(x)
        return None

In [4]:
def link_status(rvtime, page_creation_time):
    if pd.isnull(page_creation_time):
        return 'check'
    if page_creation_time > rvtime:
        return 'red link'
    if page_creation_time <= rvtime:
        return 'blue link'

### import files

In [5]:
page_creation_df = pd.read_csv('../'+'quarry-44795-get-the-timestamp-for-first-revision-of-all-articles-in-ukwiki-run466602.csv')

### Check for chosen pages

In [8]:
noise_pages = ('Головна сторінка', 'Обговорення:', 'Обговорення користувача:', 'Обговорення Вікіпедії:', 'Категорія:', 'Шаблон:', 'Вікіпедія:',
               'Файл:','Користувач:', 'Категорія:Списки:', 'Портал:', 'Довідка:')

# Construct dump file iterator
dump = Iterator.from_file(open("/media/andrii/earth/Katia/!redlinks_project/ukwiki_dumps/ukwiki-20200501-pages-meta-history1.xml-p1p17370"))

# Iterate through 20 pages

stopwords = ['[[Категорія:', '[[Файл:']
revs_links_time = []

i = 20
for page in dump:
    if i == 0:
        break
    elif page.title.startswith(noise_pages):
        continue
    elif page.redirect is not None:
        continue
    else:
        print('PAGE TITLE:', page.title)
        #print('PAGE ID:', page.id)
        #print()
        for revision in page:
            timestamp = revision.timestamp
            revision = revision.text
            get_revision = mwparserfromhell.parse(revision)
            links = get_revision.filter_wikilinks()
            filtered = []
            for l in links:
                if not any([sw in l for sw in stopwords]):
                    revs_links_time.append((page.title, str(l), timestamp))
    i -= 1

PAGE TITLE: Географія
PAGE TITLE: Атом
PAGE TITLE: Мільярд
PAGE TITLE: Ядро
PAGE TITLE: Мільйон
PAGE TITLE: Хімія
PAGE TITLE: Діаметр
PAGE TITLE: Метр
PAGE TITLE: Біологія
PAGE TITLE: Коло
PAGE TITLE: Довжина
PAGE TITLE: Точка
PAGE TITLE: Крапка
PAGE TITLE: Речення
PAGE TITLE: Слово
PAGE TITLE: Мова
PAGE TITLE: Радіус
PAGE TITLE: Есперанто
PAGE TITLE: 1944
PAGE TITLE: 1854


In [9]:
# create dataframe of all links from all revisions of the chosen pages
revision_links_df = pd.DataFrame(revs_links_time, columns=['parent_page', 'link', 'rvtimestamp'])
revision_links_df['link_title'] = revision_links_df['link'].apply(lambda row: multiple_replace(row))
revision_links_df['link_title'] = revision_links_df['link_title'].str.lower()

In [10]:
# prepare a table with page creation time
page_creation_df.page_creation_timestamp = page_creation_df.page_creation_timestamp.apply(lambda x: Timestamp(np.int64(x)))
page_creation_df['page_title'] = page_creation_df['page_title'].str.lower()

In [11]:
# create a table of links with their status (red/blue/check)
link_status_df = revision_links_df.merge(page_creation_df, left_on='link_title', right_on='page_title', how='left')
del link_status_df['page_title']
link_status_df['link_status'] = link_status_df.apply(lambda x: link_status(x['rvtimestamp'], x['page_creation_timestamp']), axis=1)

# clean noise links
noise_langlinks = ('af:', 'als:', 'ar', 'bg:', 'bs:', 'ca:', 'cs:', 'da:', 'de:', 'el:', 'en:', 'es:', 'et:', 'eu:', 'fi:', 'fr:', 'ga:',
              'hi:', 'hr:', 'hu:', 'ia:', 'is:', 'it:', 'ja:', 'ko:', 'la:', 'lt:', 'nl:', 'no:', 'oc:',
              'pl:', 'pt:', 'ro:', 'ru:', 'sk:', 'sl:', 'sq:', 'sr:', 'sv:', 'sw:', 'tr:', 'uk:', 'zh:', 'co:',
              'cy:', 'eo:', 'fy:', 'gl:', 'he:', 'id:', 'ks:', 'lv:', 'ms:', 'nds:', 'simple:', 'th:', 
              'category:', 'ast:', 'az:', 'csb:', 'ie:', 'lb:', 'te:', 'tl:', 'ur:', 'vi:', 'vo:', 'zh-cn:',
              'image:', 'be:', 'bn:', 'br:', 'io:', 'ka:', 'mg:', 'mk:', 'mt:', 'rm:', 'scn:', 'ab:', 'fo:',
              'fur:', 'li:', 'ab:', 'an:', 'ba:', 'ky:', 'nn:', 'os:', 'kv:', 'sc:', 'ceb:', 'lad:', 'mh:',
              'gu:', 'ta:', 'to:', 'tk:', 'зображення:', 'av:', 'bat-smg:', 'bm:', 'cv:', 'ku:', 'kw:', 'lmo:',
              'nrm:', 'ps:', 'rmy:', 'sh:', 'tg:', 'ug:', 'uz:', 'vec:', 'zh-yue:', 'ilo:', 'bar:', 'iu:',
              'nov:', 'wa:', 'qu:', 'wa:', 'my:', 'frp:', 'hsb:', 'nah:', 'yi:', 'ht:', 'mr:', 'ng:', 'sco:',
              'su:', 'be-x-old:', 'ne:', 'km:', 'mn:', 'sm:', 'fa:', 'lij:', 'map-bms:', 'ml:', 'ru-sib:',
              'jv:', 'wo:', 'tpi:', 'as:', 'tpi:', 'lo:', 'bh:', 'war:', 'mzn:', 'pms:', 'stq:', 'szl:',
              'ay:', 'si:', 'fiu-vro:', 'ln:', 'bcl:', 'ln:', 'dv:', 'gn:', 'pam:', 'so:', 'zea:', 'ext:', 'gan:',
              'hy:', 'sah:', 'ak:', 'kab:', 'nds-nl:', 'mdf:', 'ts:', 'gv:', 'pnt:', 'kk:', 'zh-min-nan:', 'jbo:',
              'kn:', 'kl:', 'mwl:', 'hif:', 'mhr:', 'yo:', 'new:', 'pcd:', 'tt:', 'krc:', 'am', 'frr:', 'ig:',
              'pnb:', 'bo:', 'bi:', 'ss:', 'rue:', 'kbd:', 'zh-classical:', 'sn:', 'sa:', 'roa-tara:', 'gd:', 'cbk-zam:',
              'lez:', 'nso:', 'ckb:', 'vep:', 'pa:', 'file:', 'kaa:', 'pfl:', 'haw:', 'pih:', 'mo:', 'ee:', 'xh:',
              'srn:', 'wuu:', 'ksh:', 'ki:', 'szl:', 'crh:', 'zh-tw:', 'dsb:', 'lg:', 'koi:', 'xal:', 'jp:',
              'pih:', 'hak:', ':категорія:', 'cu:', ':ru:', 'ff:', 'wikipedia:en:', 'bug:', 'xmf:', 'ce:',
              'diq:', 'vls:', 'bsd', 'myv:', 'bjn:', 'or:', 'ti:', 'cdo:', 'mi:', 'pap:', 'se:', 
               'історія_науки#', 'planetmath:')

link_status_df_clean = link_status_df.drop(link_status_df[link_status_df.link_title.astype(str).str.startswith(noise_langlinks)].index)

In [12]:
# get strange pages
check = link_status_df_clean['link_title'].values.tolist()
print('number of link titles that need checking:', len(set(check)))

number of link titles that need checking: 2317


In [13]:
# get a table only with red links
df_red = link_status_df_clean.loc[link_status_df_clean['link_status'] == 'red link']

In [15]:
# check (manually with web.archive.org) whether this method of retrieving red links is OK.
df_red.loc[df_red['parent_page'] == 'Біологія'].sample(n=50)

,parent_page,link,rvtimestamp,link_title,page_creation_timestamp,link_status
411258,Біологія,[[харчування]],20110405185524,харчування,20111209083648,red link
459637,Біологія,[[Вода|води]],20130318152132,вода,20180310051644,red link
410836,Біологія,[[20 століття]],20110402140851,20_століття,20121117105004,red link
418541,Біологія,[[Вода|води]],20110624184336,вода,20180310051644,red link
410202,Біологія,[[Біоспелеологія]],20110401200813,біоспелеологія,20140812062701,red link
464680,Біологія,[[генна інженерія|генної інженерії]],20130822112608,генна_інженерія,20160725133326,red link
418342,Біологія,[[викопні організми|викопних]],20110624184336,викопні_організми,20110716103442,red link
449503,Біологія,[[Лудольф Тревіранус]],20121023074935,лудольф_тревіранус,20200423182411,red link
491060,Біологія,[[мамаліологія]],20170106233931,мамаліологія,20170708224822,red link
398389,Біологія,[[систематика]],20100620092537,систематика,20110826173859,red link


## Results

For 100 random links of the oldest pages 46 % are correctly marked as red.

Many pages are incorrectly marked red because of redirections that were main pages that time. We should compare the time of red link revision with the oldest version among the considered page and its redirectons.